In [136]:
import os
import xarray as xr
import pandas as pd
import geopandas as gpd


In [137]:
IN_DIR = r"X:\georisk\HaRIA_B_Wind\projects\qfes_swha\data\derived\hazard\projections"

In [138]:
GROUP = "GROUP2"
RCPLIST=["RCP45","RCP85"]
PERIODS=["2021-2040","2041-2060","2061-2080","2081-2100"]
filename = os.path.join(IN_DIR, f"{GROUP}_RCP45_2081-2100/hazard", "hazard_rel.nc")
histfilename = os.path.join(IN_DIR, f"{GROUP}_RCP45_1981-2020/hazard", "hazard.nc")

In [139]:
stndf = gpd.read_file(r"X:\georisk\HaRIA_B_Wind\projects\qfes_swha\data\reference_data\SEQ_station_list.shp")
stndf = pd.DataFrame(stndf.drop("geometry", axis=1))

In [140]:
ds = xr.open_dataset(filename)
histds = xr.open_dataset(histfilename)
ari = ds.ari.values


In [141]:
outdf = pd.DataFrame(columns=["stnID", "Place", "period", "RCP", *ari,])
outdf.set_index(['stnID', "Place", "period", "RCP"], inplace=True)

In [142]:
for RCP in RCPLIST:
    histfilename = os.path.join(IN_DIR, f"{GROUP}_{RCP}_1981-2020/hazard", "hazard.nc")
    print(histfilename)
    histds = xr.open_dataset(histfilename)

    for PER in PERIODS:
        filename = os.path.join(IN_DIR, f"{GROUP}_{RCP}_{PER}/hazard", "hazard_rel.nc")
        print(filename)
        ds = xr.open_dataset(filename)
        for idx, row in stndf.iterrows():
            print(row.Place)
            df = ds.sel(lat=row.Latitude, lon=row.Longitude, method='nearest').to_dataframe()
            hdf = histds.sel(lat=row.Latitude, lon=row.Longitude, method='nearest').to_dataframe()
            relchange = df['wspd'] / hdf['wspd']
            outdf.loc[(idx, row.Place, PER, RCP),:] = 100*relchange.values.T

outdf.to_csv(os.path.join(IN_DIR, "station_hazard_change_GROUP2.csv"))
    

X:\georisk\HaRIA_B_Wind\projects\qfes_swha\data\derived\hazard\projections\GROUP2_RCP45_1981-2020/hazard\hazard.nc
X:\georisk\HaRIA_B_Wind\projects\qfes_swha\data\derived\hazard\projections\GROUP2_RCP45_2021-2040/hazard\hazard_rel.nc
AMBERLEY AMO
OAKEY AERO
ARCHERFIELD AIRPORT
TOOWOOMBA AIRPORT
COOLANGATTA
BRISBANE AERO
WARWICK
GYMPIE
UNIVERSITY OF QUEENSLAND GATTON
KINGAROY AIRPORT
TEWANTIN RSL PARK
GOLD COAST SEAWAY
DALBY AIRPORT
APPLETHORPE
BRISBANE AERO
CAPE MORETON LIGHTHOUSE
DOUBLE ISLAND POINT LIGHTHOUSE
SUNSHINE COAST AIRPORT
NAMBOUR DAFF - HILLSIDE
NAMBOUR DPI
BEERBURRUM FOREST STATION
REDCLIFFE
BRISBANE REGIONAL OFFICE
X:\georisk\HaRIA_B_Wind\projects\qfes_swha\data\derived\hazard\projections\GROUP2_RCP45_2041-2060/hazard\hazard_rel.nc
AMBERLEY AMO
OAKEY AERO
ARCHERFIELD AIRPORT
TOOWOOMBA AIRPORT
COOLANGATTA
BRISBANE AERO
WARWICK
GYMPIE
UNIVERSITY OF QUEENSLAND GATTON
KINGAROY AIRPORT
TEWANTIN RSL PARK
GOLD COAST SEAWAY
DALBY AIRPORT
APPLETHORPE
BRISBANE AERO
CAPE MORETON LIG

RuntimeError: NetCDF: HDF error

In [143]:
ds = xr.open_dataset(r"X:\georisk\HaRIA_B_Wind\projects\qfes_swha\data\derived\hazard\projections\GROUP2_RCP85_2021-2040/hazard\hazard_rel.nc")

In [144]:
ds

<xarray.Dataset>
Dimensions:  (ari: 14, lon: 1251, lat: 1251)
Coordinates:
  * ari      (ari) float32 2.0 5.0 10.0 20.0 25.0 ... 1e+03 2e+03 5e+03 1e+04
  * lon      (lon) float32 135.0 135.0 135.0 135.1 ... 159.9 160.0 160.0 160.0
  * lat      (lat) float32 -5.0 -5.02 -5.04 -5.06 ... -29.94 -29.96 -29.98 -30.0
Data variables:
    crs      float64 ...
    wspd     (ari, lat, lon) float32 ...
Attributes: (12/106)
    CDI:                                Climate Data Interface version 1.9.8 ...
    history:                            Mon Sep 14 13:50:19 2020: cdo -L -P 4...
    Conventions:                        CF-1.6
    title:                              TCRM hazard simulation
    tcrm_version:                       /g/data/w85/software/tcrm/master/tcrm...
    python_version:                     3.7.4 (default, Nov  6 2019, 12:34:08...
    ...                                 ...
    TCLV_dateformat:                    %Y-%m-%d %H:%M:%S
    TCLV_speedunits:                    kph
    created_on:                         2020-09-11 11:02:14
    created_by:                         cxa547
    CDO:                                Climate Data Operators version 1.9.8 ...
    cdo_openmp_thread_number:           48